### **SLAM** (Simultaneous Localization and Mapping)

SLAM is a technique used in robotics and computer vision to create a **`map`** of an unknown environment while simultaneously keeping track of the **`agent's location`** within that environment. It combines data from various sensors, such as cameras and LiDAR, to build a 2D (grid maps) or 3D map (3D point cloud) and localize the robot within it.

Two main techniques used in SLAM are:
1. Build a map of an unknown environment (mapping).
2. Track its own pose (position and orientation) within that map at the same time (localization).

Core functionalities of Slam Algorithms:

1. `Sensor inputs`
- SLAM typically uses sensor data (e.g., LIDAR scans, camera images, or depth sensor measurements) to detect features or landmarks in the environment.
2. `State estimation`
- An internal state (the robot’s pose, including x, y, yaw) is estimated using algorithms like Extended Kalman Filters, Particle Filters, or Graph Optimization.
3. `Map building`
- As the robot moves, it accumulates new sensor data. The SLAM algorithm integrates that data into a global map (2D grid map, 3D point cloud, or other representations).
4. `Loop closure`
- When the robot revisits a previously mapped area, the SLAM algorithm detects that it’s the same place (loop closure). This knowledge is used to reduce accumulated drift and refine both the map and pose estimates.





Slam toolbox:

In [ ]:
sudo apt install ros-jazzy-slam-toolbox

There is a new node in the launch file marker_server from the `interactive-marker-twist-server` this can be used to `move and rotate our robot directly from **RViz***`. Let's install it with:

In [ ]:
sudo apt install ros-jazzy-interactive-marker-twist-server

We moved all the `parameter_bridge` topics into a yaml config file `gz_bridge.yaml`, we don't describe them in the launch file anymore, this is more comfortable when we forward many topics

`base_link` → robot’s body center.

`odom` frame → comes from wheel encoders (`local` odometry frame).
- It estimates how much your `robot moved` — but it `drifts` over time (small errors add up).

`map` frame → comes from SLAM (e.g. from LiDAR-based mapping, `Global` reference frame).
- It constantly `corrects` that drift by `matching sensor data with the map`.


So, if your odometry drifts, the SLAM algorithm shifts the robot’s map pose to keep it consistent with real-world features.
The difference between map and odom = how much drift your raw odometry accumulated since SLAM had to correct it.



---
#### **MAPPING**


SLAM tool box offers 2 ways to store maps:

1. Save Map  
- Command: `ros2 run nav2_map_server map_saver_cli -f my_map`  
- Output:  
  - `my_map.pgm` → grayscale image (white = free, black = occupied)  
  - `my_map.yaml` → metadata (resolution, origin, etc.)  
- Use: for localization only later (e.g. `nav2_amcl`)  
- Limitation: cannot resume mapping since it's just an image  
- Analogy: like taking a screenshot of your map  

2. Serialize Map  
- Command: `ros2 service call /slam_toolbox/serialize_map ...`  
- Saves SLAM Toolbox’s internal graph (poses, scans, loop closures, etc.)  
- Allows later deserialization to continue mapping from the same state  
- Use: for continuing SLAM sessions, not for localization with other nodes  
- Analogy: like saving a project file to edit later  

**Note:** In Gui windows type the name of both save and serialize files, and click once , they will be automatically saved in the *_ws folder, we must be in the *_ws folder during launch file.

3. Deserialize Map  
- Command: `ros2 service call /slam_toolbox/deserialize_map ...`
- Loads a previously serialized map to resume SLAM
- Use: to continue mapping from where you left off
- Limitation: cannot use deserialized maps for localization with other nodes
- Analogy: like opening a saved project file to continue working

**Note:** Type name of any saved serialize_map without any extension (both .data and .posegraph files should be in the *_ws folder) in the Gui window and click once, it will be automatically loaded.



---

#### **Custom deserialize map executable Working**

1. MapLoaderNode which calls the service to deserialize:

- Doesn’t open or parse the map itself. Calls a ROS2 service:
`/slam_toolbox/deserialize_map`
(service type slam_toolbox/srv/DeserializePoseGraph).

- That service is implemented inside the slam_toolbox node, not by us. So, when our node sends this request:
`self.request.filename = "/root/nav_ws/install/ros2_nav/share/ros2_nav/maps/serialized"`

- the slam_toolbox node receives it, opens that serialized file (a .posegraph file), and reconstructs:
   - its internal pose graph,
   - the map data,
   - and all the robot poses stored inside it.

- Essentially, slam_toolbox “loads” the map into its running memory, not into another file.

2. Once slam_toolbox finishes deserializing:

- It populates its internal occupancy grid, constraints, and graph nodes.
- Then `it starts advertising topics` like:
   - /map (OccupancyGrid)
   - /map_metadata
   - /slam_toolbox/pose_graph_visualization
   - /slam_toolbox/map_updates
- It also provides the /map -> /odom transform via TF2.

That’s the `shared state` that `other launch files` (like navigation or localization launch) use. they don’t reload the serialized file themselves.
Other `launch files/nodes` just `connect to those ROS2 topics/services`.

---

#### **LOCALIZATION**

While mapping was the process of creating a representation (a map) of an environment. Localization is the process by which a `robot determines its own position and orientation within a known environment (map)`. In other words:

- The environment or map is typically already available or pre-built.
- The robot’s task is to figure out “Where am I?” or “Which direction am I facing?” using sensor data, often by `matching its current perceptions to the known map`.


#### Localization with AMCL
`AMCL` (Adaptive Monte Carlo Localization) is a `particle filter–based 2D localization algorithm`. The robot’s possible `poses` (position + orientation in 2D) are represented by a `set of particles`. It adaptively samples the robot’s possible poses according to sensor readings and motion updates, converging on an accurate estimate of where the robot is within a known map.

AMCL is part of the ROS2 navigation stack, let's install it first:

In [ ]:
sudo apt install ros-jazzy-nav2-bringup 
sudo apt install ros-jazzy-nav2-amcl

Write a separate launch file for localization, name it `localization.launch.py` in the `launch` folder of your package. include rviz_config_arg, rviz_launch_arg, sim_time_arg, nav2_localization_launch_path (Nav2 bringup),  localization_params_path(amcl_localization.yaml), map_file_path (my_map.yaml), rviz_node, sim_time_arg,    interactive_marker_twist_server_node, localization_launch.


[amcl_localization.yaml](https://docs.nav2.org/configuration/packages/configuring-amcl.html) (Link from official doc of Nav2). We have to set this for our localization.launch.py file.

[Nav2 Documentation](https://docs.nav2.org/index.html)

The `main purpose` of the `localization algorithm` is establishing the `transformation` between the `fixed map` and the `robot's odometry frame` based on `real time sensor data`. We can visualize this in RViz as we saw it during mapping.

---


#### Correlation vs Covariance 
- Correlation describe the direction and strength of relationship between two Random variables.
- Covariance indicates the extent/direction to which two random variables change together.

<img src="assets/c1.png" width="800" height="600"/>


**`Population`**
All possible data points in a group you want to study.
Example: the heights of all students in a university.

**`Sample`**
A smaller subset taken from the population to estimate its characteristics.
Example: heights of 50 students selected from that university.

<img src="assets/c2.png" width="750" height="600"/>




<img src="assets/c3.png" width="375" height="200"/>
<img src="assets/c4.png" width="375" height="200"/>

<img src="assets/c5.png" width="750" height="600"/>

---

<img src="assets/c6.png" width="750" height="300"/>
<br>
<img src="assets/c7.png" width="750" height="500"/>

<img src="assets/c8.png" width="600" height="200"/>
<br>
<img src="assets/c9.png" width="600" height="550"/>
<br>

**`Variance`**
- Measures how far each data point deviates from the mean on average (spread of data).
- Example: if exam scores are 70, 80, 90, the variance shows how far each score lies from the mean 80.

**`Standard deviation`**
- It is the square root of variance, giving spread in the same unit as the data.
- Example: if variance of exam scores is 25, then standard deviation is 5, meaning scores typically differ by 5 marks from the mean.

<img src="assets/c10.png" width="600" height="450"/>

---


#### Covariance Matrix:

<img src="assets/c11.png" width="750" height="600"/>
<br>
<img src="assets/c13.png" width="750" height="600"/>


<img src="assets/c14.png" width="750" height="300"/>

---


#### **Pose Estimation**
- Initial Pose Calculation of Robot using Covariance Matrix


AMCL needs the `initial pose` which can be given using `RviZ's` `2D Estimate Pose` tool. Then AMCL's particles start appearing on the RviZ, each particle represents a pose [x,y,yaw(0)]. 

We can define our own Custom node for `initial pose sending` and run using `ros2 run ros2_nav_py send_initial_pose` which will be a publisher node and publish at `/initialpose` which is required by AMCL

<img src="assets/c15.png" width="750" height="550"/>
<img src="assets/c16.png" width="750" height="550"/>
<img src="assets/c17.png" width="750" height="600"/>
<img src="assets/c18.png" width="750" height="600"/>

----

#### **Eulear Angles (rpy) vs Quaternions (qx,qy,qz,qw)**



<img src="assets/q2.png" width="750" height="500"/>

<img src="assets/q6.png" width="750" height="500"/>

<img src="assets/q3.png" width="750" height="600"/>

<img src="assets/q1.png" width="750" height="600"/>
<img src="assets/q4.png" width="750" height="500"/>
<img src="assets/quaternions.png" width="750" height="500"/>
<img src="assets/q5.png" width="750" height="600"/>

- `Euler angles` use `sequential rotations` and suffer from `singularities` at specific configurations.
- `Quaternions` use a `unified 4D representation` based on axis–angle formulation, preserving smoothness and numerical stability across all possible orientations.

Links: (also in assets folder)

[Eulear Angles](https://danceswithcode.net/engineeringnotes/rotations_in_3d/rotations_in_3d_part1.html)

[Quaternions](https://danceswithcode.net/engineeringnotes/quaternions/quaternions.html)

---

#### **Localization with SLAM**

It's possible to use SLAM toolbox in `localization mode`, it requires a small adjustment on the parameters which is already part of this lesson, slam_toolbox_localization.yaml. This requires the path to the serialized map file.Make sure the map_file_name parameter is changed to the path on your machine to the serialized map file! Create the launch file for localization, localization_slam_toolbox.launch.py, it's very similar to the SLAM toolbox mapping launch file.

Rebuild the workspace and try it, we'll see that the map keeps updating unlike with AMCL, this is the normal behavior of the localization with SLAM toolbox. According to this [paper](https://joss.theoj.org/papers/10.21105/joss.02783), the localization mode does continue to update the pose-graph with new constraints and nodes, but the updated map expires over some time. SLAM toolbox describes it as "elastic", which means it holds the updated graph for some amount of time, but does not add it to the permanent graph.

----

### **Navigation**
Navigation in robotics is the overall process that enables a robot to move from one location to another in a safe, efficient, and autonomous manner. It typically involves:

- Knowing where the robot is (`localization or SLAM`),
- Knowing where it needs to go (a `goal pose or waypoint`),
- Planning a path to reach that goal (`path planning`), and
- Moving along that path while avoiding dynamic and static obstacles (`motion control` and `obstacle avoidance`).

ROS's nav2 navigation stack implements the above points 2 to 4, for the first point we already met several possible solutions.

---


#### **Navigation with SLAM**

As we saw to navigate a mobile robot we need to know 2 things:

- Knowing where the robot is (localization or SLAM),
- Knowing where it needs to go (a goal pose or waypoint)
In the previous examples we used `localization on a known map`, but it's also possible to navigate together with an `online SLAM`. It means we don't know the complete environment around the robot but we can already navigate in the known surrounding.

Create a navigation_with_slam.launch.py launch file where we start both the `SLAM toolbox` and the `navigation stack`.

---

#### **Exploration**
Exploration is a process by which a robot operating in an `unknown` or partially known `environment` actively `searches` the space to gather new information. This is a real life use case to use SLAM together with the navigation stack.

```bash
git clone https://github.com/MOGI-ROS/m-explore-ros2.git
```
Build the workspace and source the setup.bash to make sure ROS is aware about the new package!

We'll need 3 terminals, one for the simulation:
```bash
ros2 launch bme_ros2_navigation spawn_robot.launch.py
```
In another terminal we launch the navigation_with_slam.launch.py:
```bash
ros2 launch bme_ros2_navigation navigation_with_slam.launch.py
```
And in the third one we launch the exploration:
```bash
ros2 launch explore_lite explore.launch.py
```
The exploration node will identify the boundaries of the know surrounding and will navigate the robot until it finds all the physical boundaries of the environment.

---





Custom camera_subscriber and robot_mover nodes


Main Topics and Nodes:


Use commands: 
- `ros2 topic list` to list all topics.
- `ros2 topic list -t | grep </camera>` to list all topics of named /camera (filtered output).
- `ros2 topic info <topic_name> -v ` to get details about a specific topic with verbose output.
- `ros2 topic echo <topic_name> --once` to see the messages being published on a topic in real-time only once.



1. **`/camera/image`**

- *Publisher*: Usually a camera driver node (like realsense_camera, usb_cam, gazebo_ros_camera, or image_publisher, `ros_gz_image` (This node is part of the Gazebo–ROS bridge system. It takes simulated camera frames from Gazebo and publishes them into ROS2 as sensor_msgs/msg/Image.)).
- *Message type*: sensor_msgs/msg/Image → raw pixel array + metadata.
- *Subscribers*: Vision or navigation nodes (rviz2, object detector, visual SLAM, etc.).
- *Used for*: Perception, object detection, mapping, SLAM, etc.


2. **`/camera/image/compressed`**
- *Publisher*: Usually a camera driver node (like realsense_camera, usb_cam, gazebo_ros_camera, or image_publisher, `ros_gz_image` (This node is part of the Gazebo–ROS bridge system. It takes simulated camera frames from Gazebo and publishes them into ROS2 as sensor_msgs/msg/Image.)).
- *Message type*: sensor_msgs/msg/CompressedImage → JPEG compressed image data.
- *Subscribers*: Vision or navigation nodes (`rviz2` (here subscribe to it), object detector, visual SLAM, etc.).
- *Used for*: Perception, object detection, mapping, SLAM, etc.


3. **`/cmd_vel`**

- *Publisher*: Something that generates motion commands could be:
  - `teleop_twist_keyboard` (manual control), `collision_monitor`, `docking_server`
  - nav2_controller or `bt_navigator` (autonomous navigation)
- *Message type*: geometry_msgs/msg/Twist
  - linear.x → forward/back speed (m/s). `Its not distance but speed.`
  - angular.z → rotation speed (rad/s)
- *Subscriber*: The robot’s base controller (e.g., diff_drive_controller, cmd_vel_mux, or ros2_control hardware interface).
- Connection flow: `teleop_twist_keyboard → /cmd_vel → ros_gz_bridge → Gazebo robot base plugin`


In [ ]:
# run launch file to view camera frames in window and save after 30secs
ros2 launch camera_access_pkg camera_subscriber.launch.py use_sim_time:=false

---

#### Yollo v-11 in Gazebo

Magic command:
`find install -type f -executable -exec sed -i '1s|^#!.*python3$|#!/root/nav_ws/venv/bin/python3|' {} +`
- Finds every executable file in install/.
- Rewrites the first line (#!...python3) to point to your venv interpreter.
- Fixes all ROS 2 entry points to use your venv’s Python.


In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import CompressedImage, Image
from cv_bridge import CvBridge
import cv2
import torch
from ultralytics.nn import tasks
import numpy as np
from ultralytics import YOLO
import supervision as sv

class YoloSegmentation(Node):
    def __init__(self):
        super().__init__('segmentation_node')
        
        # --- Model and Annotator Initialization ---
        self.get_logger().info("Loading YOLOv11 model...")

        self.model = YOLO("yolo11n-seg.pt")
        self.mask_annotator = sv.MaskAnnotator(color=sv.ColorPalette.ROBOFLOW)
        self.box_annotator = sv.BoxCornerAnnotator(color=sv.ColorPalette.ROBOFLOW)
        self.label_annotator = sv.LabelAnnotator()
        self.get_logger().info("Model loaded successfully.")

        # --- ROS2 Communication Setup ---
        self.bridge = CvBridge()
        self.subscription = self.create_subscription(
            CompressedImage,
            '/camera/image/compressed',
            self.image_callback,
            10)
        
        self.publisher = self.create_publisher(Image, '/yolo/segmented_image', 10)
        
    def image_callback(self, msg: CompressedImage):
        try:
            # 1. Decompress ROS message to OpenCV frame
            np_arr = np.frombuffer(msg.data, np.uint8)
            frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

            # 2. Perform inference
            results = self.model(frame)[0]
            detections = sv.Detections.from_ultralytics(results)

            # 3. Create labels
            labels = [
                f"{self.model.names[class_id]} {confidence:0.2f}"
                for confidence, class_id
                in zip(detections.confidence, detections.class_id)
            ]
            
            # 4. Annotate the frame
            annotated_frame = self.mask_annotator.annotate(scene=frame.copy(), detections=detections)
            annotated_frame = self.box_annotator.annotate(scene=annotated_frame, detections=detections)
            annotated_frame = self.label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

            # 5. Convert annotated frame back to ROS message and publish
            annotated_msg = self.bridge.cv2_to_imgmsg(annotated_frame, "bgr8")
            self.publisher.publish(annotated_msg)

        except Exception as e:
            self.get_logger().error(f"Error processing image: {e}")

def main(args=None):
    rclpy.init(args=args)
    node = YoloSegmentation()
    rclpy.spin(node)
    node.destroy_node()
    rclpy.shutdown()

if __name__ == '__main__':
    main()

In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import CompressedImage, Image
from rclpy.qos import qos_profile_sensor_data
from cv_bridge import CvBridge
from ament_index_python.packages import get_package_share_directory
import os
import cv2
import numpy as np
import onnxruntime as ort
import supervision as sv
import time

class YoloSegmentation(Node):
    def __init__(self):
        super().__init__('segmentation_node')
        self.get_logger().info("Loading YOLOv11 ONNX model (CPU optimized)...")

        # --- ONNX Inference Session (CPU) ---
        # Get absolute path inside installed package
        pkg_path = get_package_share_directory('yollo_seg')
        model_path = os.path.join(pkg_path, 'models', 'yolo11n-seg.onnx')
        self.session = ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])
        self.input_name = self.session.get_inputs()[0].name
        self.output_names = [o.name for o in self.session.get_outputs()]
        self.get_logger().info("ONNX model loaded successfully.")

        # --- ROS2 Communication ---
        self.bridge = CvBridge()
        self.processing = False  # skip-frame flag

        self.subscription = self.create_subscription(
            CompressedImage,
            '/camera/image/compressed',
            self.image_callback,
            qos_profile_sensor_data)  # drop old frames automatically

        self.publisher = self.create_publisher(Image, '/yolo/segmented_image', 10)

        # --- Annotators (for visualization only) ---
        self.mask_annotator = sv.MaskAnnotator(color=sv.ColorPalette.ROBOFLOW)
        self.box_annotator = sv.BoxCornerAnnotator(color=sv.ColorPalette.ROBOFLOW)
        self.label_annotator = sv.LabelAnnotator()

    def image_callback(self, msg: CompressedImage):
        if self.processing:
            return  # skip if still busy
        self.processing = True
        start_time = time.time()

        try:
            # --- 1. Decompress image ---
            np_arr = np.frombuffer(msg.data, np.uint8)
            frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

            # --- 2. Preprocess: resize & normalize ---
            resized = cv2.resize(frame, (320, 320))  # smaller = faster
            img = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
            img = img.astype(np.float32) / 255.0
            img = np.transpose(img, (2, 0, 1))  # HWC → CHW
            img = np.expand_dims(img, axis=0)

            # --- 3. Inference ---
            outputs = self.session.run(self.output_names, {self.input_name: img})

            # TODO: decode segmentation (optional — based on your exported model)
            # For now, we’ll just republish a resized grayscale placeholder mask
            seg_output = (np.mean(resized, axis=2)).astype(np.uint8)

            annotated_msg = self.bridge.cv2_to_imgmsg(seg_output, "mono8")
            self.publisher.publish(annotated_msg)

            dt = time.time() - start_time
            self.get_logger().info(f"Frame processed in {dt:.3f}s ({1/dt:.2f} FPS)")

        except Exception as e:
            self.get_logger().error(f"Error processing image: {e}")
        finally:
            self.processing = False


def main(args=None):
    rclpy.init(args=args)
    node = YoloSegmentation()
    rclpy.spin(node)
    node.destroy_node()
    rclpy.shutdown()


if __name__ == '__main__':
    main()


Yollo-11E

In [ ]:
#!/usr/bin/env python3
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import CompressedImage, Image
from cv_bridge import CvBridge
import cv2
import numpy as np
from ultralytics import YOLOE  # <-- YOLOE import (new)
import supervision as sv       # for visualization helpers


class YoloESegmentation(Node):
    def __init__(self):
        super().__init__('yoloe_segmentation_node')

        # --- Load YOLOE model ---
        self.get_logger().info("Loading YOLOE-11S prompt-free segmentation model...")
        # use smallest, prompt-free version for best performance on edge
        self.model = YOLOE("yoloe-11s-seg-pf.pt")  
        self.get_logger().info("Model loaded successfully.")

        # --- Visualization annotators (for masks, boxes, labels) ---
        self.mask_annotator = sv.MaskAnnotator(color=sv.ColorPalette.ROBOFLOW)
        self.box_annotator = sv.BoxCornerAnnotator(color=sv.ColorPalette.ROBOFLOW)
        self.label_annotator = sv.LabelAnnotator()

        # --- ROS setup ---
        self.bridge = CvBridge()

        # Subscribe to compressed image topic
        self.subscription = self.create_subscription(
            CompressedImage,
            '/camera/image/compressed',
            self.image_callback,
            10
        )

        # Publish segmented annotated image
        self.publisher = self.create_publisher(Image, '/yolo/segmented_image', 10)

        self.get_logger().info("Subscribed to /camera/image/compressed")

    def image_callback(self, msg: CompressedImage):
        try:
            # 1. Convert ROS CompressedImage → OpenCV image
            np_arr = np.frombuffer(msg.data, np.uint8)
            frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

            if frame is None:
                self.get_logger().warn("Received empty frame, skipping.")
                return

            # 2. Run YOLOE inference (prompt-free segmentation)
            results = self.model.predict(frame, verbose=False)[0]

            # 3. Convert YOLO output → Supervision detections
            detections = sv.Detections.from_ultralytics(results)

            # 4. Prepare display labels
            labels = [
                f"{self.model.names[class_id]} {confidence:.2f}"
                for confidence, class_id in zip(detections.confidence, detections.class_id)
            ]

            # 5. Annotate frame with masks + boxes + labels
            annotated = frame.copy()
            annotated = self.mask_annotator.annotate(scene=annotated, detections=detections)
            annotated = self.box_annotator.annotate(scene=annotated, detections=detections)
            annotated = self.label_annotator.annotate(scene=annotated, detections=detections, labels=labels)

            # 6. Convert back to ROS Image message and publish
            img_msg = self.bridge.cv2_to_imgmsg(annotated, encoding="bgr8")
            self.publisher.publish(img_msg)

        except Exception as e:
            self.get_logger().error(f"Error processing frame: {e}")


def main(args=None):
    rclpy.init(args=args)
    node = YoloESegmentation()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        node.get_logger().info("Shutting down YOLOE segmentation node.")
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == "__main__":
    main()


---

#### Modelling actors in Gazebo


In [ ]:

<actor name="actor_walking">
  <!--basic minimal starting code-->
  <!-- Starting pose, nice for when the world is reset -->
  <!--this pose is different from next one in waypoints, so set this first then next-->
  <pose>
    -5.5
    0.0
    0.0
    0.0
    0.0
    0.0
  </pose>
  <!-- Actor visual model -->
  <skin>
    <filename>package://ros2_nav/worlds/models/walk.dae</filename>
    <scale>1.0</scale>
  </skin>
</actor>

In [ ]:
<actor name="actor_walking">
<!-- Starting pose, nice for when the world is reset -->
  <!--this pose is different from next one, so set this first then next-->
  <pose>
    -5.5
    0.0
    0.0
    0.0
    0.0
    0.0
  </pose>
  <!-- Actor visual model -->
  <skin>
    <filename>package://ros2_nav/worlds/models/walk.dae</filename>
    <scale>1.0</scale>
  </skin>
  <!-- Actor animation -->
  <animation name="walk">
    <filename>package://ros2_nav/worlds/models/walk.dae</filename>
    <interpolate_x>true</interpolate_x>
  </animation>
    <script>
      <loop>true</loop>
      <delay_start>0.000000</delay_start>
      <auto_start>true</auto_start>
      <!--type must match the animation name-->
      <trajectory id="0" type="walk">
      <!--go straight, turn 360 and then comeback in reverse-->
      <waypoint>
        <time>0</time>
        <pose>-1 0 1.0 0 0 0</pose>
      </waypoint>
      <waypoint>
        <time>2</time>
        <pose>1 0 1.0 0 0 0</pose>
      </waypoint>
      <waypoint>
        <time>4</time>
        <pose>3 0 1.0 0 0 0</pose>
      </waypoint>
      <waypoint>
        <time>6</time>
        <pose>5 0 1.0 0 0 0</pose>
      </waypoint>
      <waypoint>
        <time>8</time>
        <pose>7 0 1.0 0 0 0</pose>
      </waypoint>
      <waypoint>
        <time>8.5</time>
        <pose>7 0 1.0 0 0 3.1416</pose>
      </waypoint>
      <waypoint>
        <time>10.5</time>
        <pose>5 0 1.0 0 0 3.1416</pose>
      </waypoint>
      <waypoint>
        <time>12.5</time>
        <pose>3 0 1.0 0 0 3.1416</pose>
      </waypoint>
      <waypoint>
        <time>14.5</time>
        <pose>1 0 1.0 0 0 3.1416</pose>
      </waypoint>
      <waypoint>
        <time>16.5</time>
        <pose>-1 0 1.0 0 0 3.1416</pose>
      </waypoint>
      <waypoint>
        <time>16.5</time>
        <pose>-1 0 1.0 0 0 0</pose>
      </waypoint>
      </trajectory>
  </script>
</actor>

In [ ]:
#1. Rebuild to ensure correct permissions
colcon build --packages-select yollo_seg --symlink-install

#2. Source your workspace
source install/setup.bash

#3. If still failing, manually add execute permission
chmod +x install/yollo_seg/lib/yollo_seg/segmentation_node

#4. Verify it’s executable
ls -l install/yollo_seg/lib/yollo_seg/segmentation_node

#5. Relaunch
ros2 launch yollo_seg yollo_seg.launch.py


---

#### New models in .sdf files


- check using: `ls /root/gz_sim/gazebo_models`
- check using: `echo $GZ_SIM_RESOURCE_PATH`

- See any breaks in .sdf file using: 
`xmllint /root/nav_ws/src/ros2_nav/worlds/new.sdf --noout`


- Every time we save a gazebo file as world: Replace this using Ctrl+f with nothing:
file:///root/nav_ws/install/ros2_nav/share/ros2_nav/worlds/

- Remove this at end of the file, when added new models:


In [ ]:
<uri>file://<urdf-string></uri> 
<name>mogi_bot</name> 
<pose>2.5000000000756399 1.500000000172296 0.09999915204843178 8.2075346820849609e-10 1.8688510693122136e-09 -1.570699999999998</pose> 
</include> <include> <uri>file:///root/gz_sim/gazebo_models/Table</uri> 
<name>Table</name> 
<pose>4.5043991416676885 -5.1928294349772566 0 0 0 0</pose> 
</include> 
<include> 
<uri>file:///root/gz_sim/gazebo_models/Office Chair</uri> 
<name>OfficeChair</name> 
<pose>4.7650436507482272 -6.3532852596903684 0 0 0 0</pose> 
</include>

---



Wall with scars of damage best

![image](assets/wall_damage_scars.png)

In [ ]:
<material>
  <pbr>
    <metal>
      <!-- Path to your downloaded texture -->
      <albedo_map>package://ros2_nav/worlds/models/wall_gray_bullets.jpg</albedo_map>
      <roughness>0.8</roughness>   <!-- matte finish -->
      <metalness>0.05</metalness>   <!-- low reflectivity -->
    </metal>
  </pbr>

  <!-- Stone-gray tone: cool gray with hint of beige/olive -->
  <ambient>0.40 0.41 0.38 1</ambient>
  <diffuse>0.45 0.46 0.43 1</diffuse>
  <specular>0.10 0.10 0.09 1</specular>
  <emissive>0 0 0 1</emissive>
</material>

- Magic Command: Issues and breaking of .sdf/.urdf file can be checked using:
syntax:

    `xmllint --noout --format </path>`

    xmllint --noout --format /home/ah/nav_ws/install/ros2_nav/share/ros2_nav/worlds/ugv_world.sdf


In [ ]:
xmllint --noout --format /home/ah/nav_ws/install/ros2_nav/share/ros2_nav/worlds/ugv_world.sdf

 Add mass and inertial properties for physics

In [ ]:
<!-- Add inertial properties for physics-->
<model name="walkie_talkie_2">
      <pose>4.099660 -4.977570 1.139800 1.452840 -1.361920 -3.091190</pose>
      <static>false</static>
      <link name="body">
        <inertial>
          <mass>0.7</mass>
          <inertia>
            <ixx>0.002</ixx>
            <ixy>0</ixy>
            <ixz>0</ixz>
            <iyy>0.002</iyy>
            <iyz>0</iyz>
            <izz>0.003</izz>
          </inertia>
          <pose>0 0 0 0 0 0</pose>
        </inertial>

        <visual name="walkie_talkie_visual">
          <geometry>
            <mesh>
              <uri>file://models/walkie-talkie/source/walkyTalky 3/walkie_talkie.dae</uri>
              <scale>0.0003 0.0003 0.0003</scale>
            </mesh>
          </geometry>
        </visual>

        <collision name="walkie_talkie_collision">
          <geometry>
            <mesh>
              <uri>file://models/walkie-talkie/source/walkyTalky 3/walkie_talkie.dae</uri>
              <scale>0.0003 0.0003 0.0003</scale>
            </mesh>
          </geometry>
        </collision>
        <self_collide>false</self_collide>
        <enable_wind>false</enable_wind>
        <kinematic>false</kinematic>
      </link>
    </model>

<!-- Add mass so Gazebo physics can simulate fall -->
<model name="gernade_4">
      <pose>-2.02990 3.872050 0.035452 -1.292830 0.607715 -1.714630</pose>
      <static>false</static>
      <link name="body">
        <!-- Add mass so Gazebo physics can simulate fall -->
        <inertial>
          <mass>0.2</mass>
          <inertia>
            <ixx>0.002</ixx>
            <iyy>0.002</iyy>
            <izz>0.002</izz>
          </inertia>
        </inertial>
        <visual name="gun_visual">
          <geometry>
            <mesh>
              <uri>file://models/gernade.dae</uri>
              <scale>1 1 1</scale>
            </mesh>
          </geometry>
        </visual>
        <collision name="gun_collision">
          <geometry>
             <mesh>
              <uri>file://models/gernade.dae</uri>
              <scale>1 1 1</scale>
            </mesh>
          </geometry>
        </collision>
      </link>
    </model>

#### Topics for robots position

In [ ]:
# local

ros2 topic echo /odometry/filtered

ros2 run tf2_ros tf2_echo map base_link 
# is not a topic — it’s a command-line tool that queries the /tf topic in real time and prints the transform between map and base_link.




In [ ]:
import rclpy
from rclpy.node import Node
from tf2_ros import Buffer, TransformListener
rclpy.init()
node = Node('tf_listener')
tf_buffer = Buffer(); tf_listener = TransformListener(tf_buffer, node)
t = tf_buffer.lookup_transform('map', 'base_link', rclpy.time.Time())  # same as tf2_echo
print(t.transform.translation)  # robot position in map frame
# better to write a node that prints this or directly include in some node


#### Blender

https://snapcraft.io/blender

In [ ]:
#Fast build and .gitignore file

# MacOS system files
**/.DS_Store

# Virtual Environments
**/venv/

# Windows system files
**/Thumbs.db

# VSCode settings
.vscode/

# Python cache files
__pycache__/
*.pyc
*.pyo
*.pyd

# ROS2 workspace build artifacts
build/
install/
log/

# Ignore recursively inside any subfolder
**/build/
**/install/
**/log/

# CMake generated files
CMakeFiles/
CMakeCache.txt
Makefile
*.cmake

# Other temporary files
*.swp
*~


# 
# sudo apt install ros-jazzy-tf-transformations
# pip install --upgrade pip setuptools colcon-common-extensions
# remove or comment     #tests_require=['pytest'], this line in setup.py of each package
# sudo apt install ros-jazzy-interactive-marker-twist-server
# git clone https://github.com/MOGI-ROS/m-explore-ros2.git
# sudo apt update
# sudo apt install python3-pyqt5
# sudo apt install ros-jazzy-nav2-msgs ros-jazzy-nav2-bringup
# sudo apt install ros-jazzy-nav2-route


----
Commands for UGV sim



In [ ]:
cd ~
source /opt/ros/jazzy/setup.bash
cd nav_ws
source ~/nav_ws/install/setup.bash
colcon build --symlink-install

# In terminal 1: Run Gazebo with the UGV world
ros2 run ros2_nav spawn_robot.launch.py world:=ugv_world.sdf

# new terminal 2: Run teleop to control the robot
source /opt/ros/jazzy/setup.bash
ros2 run teleop_twist_keyboard teleop_twist_keyboard

colcon list # to see all packages in workspace
ros2 pkg list | grep vision # to see if vision package is built and available

# new terminal 3: vision launch 
cd ~/nav_ws
source /opt/ros/jazzy/setup.bash
source ~/nav_ws/install/setup.bash
colcon build --symlink-install --packages-select vision
ros2 launch vision vision.launch.py

# new terminal 4: To view camera frames in window
rqt
# In rqt, go to Plugins -> Visualization -> Image View, then select the topic /detection/image_3d to view the camera feed.
# similar for 2nd image topic (/camera/depth_image), grab the title bar, drag it to right edge of the other image view. When we see blue docking highlight, release to review side by side.

# new terminal 5: To capture camera frames for genai_ws tasks
cd ~/genai_ws
source /opt/ros/jazzy/setup.bash
source install/setup.bash
colcon build --symlink-install --packages-select pre_processing
ros2 launch pre_processing preprocessing.launch.py

# new terminal 6: To run the field observer agent
cd ~/genai_ws
source /opt/ros/jazzy/setup.bash
source install/setup.bash
source venv/bin/activate

cd src/agentc
python3 fo_agent.py



